In [1]:
import PIL
import cv2
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
url="C:\Code\Vegetable Images\Vegetables"

In [3]:
sd=pathlib.Path(url)

In [4]:
print(sd)

C:\Code\Vegetable Images\Vegetables


In [5]:
string=str(sd)

In [6]:
string

'C:\\Code\\Vegetable Images\\Vegetables'

In [8]:
ver=[]
for i in sd.glob("*"):
    j=str(i)
    ver.append(j.replace(string+"\\",""))
print(ver)

['Bean', 'Bitter Gourd', 'Bottle Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Chilli', 'Cluster Beans', 'Corn', 'Cucumber', 'Drum Sticks', 'Ivy-Gourd', 'Lettuce', 'Okra', 'Onion', 'Papaya', 'Peas', 'Potato', 'Pumpkin', 'Radish', 'Ridge Gourd', 'Spinach', 'Tomato']


In [9]:
label_dic={}
for f in range(len(ver)):
    label_dic[ver[f]]=f

In [10]:
dic={}
for k in ver:
    dic[k]=list(sd.glob(k+"/*"))

In [11]:
tot=0
for m in ver:
    tot=tot+len(dic[m])
print(tot)

15026


In [12]:
x,y=[],[]
for veg_name,image in dic.items():
    for imag in image:
        j=cv2.imread(str(imag))
        resize=cv2.resize(j,(180,180))
        x.append(resize)
        y.append(label_dic[veg_name])

In [13]:
x=np.array(x)
y=np.array(y)

In [14]:
y=tf.keras.utils.to_categorical(y,26,dtype="float32")

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=12)

In [16]:
resize_and_rescale = tf.keras.Sequential([
  keras.layers.experimental.preprocessing.Resizing(180, 180),
  keras.layers.experimental.preprocessing.Rescaling(1./255),
])

In [17]:
data_augmentation = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.RandomFlip("horizontal",
                                                 input_shape=(180,
                                                              180,
                                                              3)),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [18]:
model=tf.keras.Sequential([
    resize_and_rescale,
    data_augmentation,
    keras.layers.Conv2D(filters=12,kernel_size=(3,3),padding="same",activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=34,kernel_size=(3,3),padding="same",activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=62,kernel_size=(3,3),padding="same",activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=86,kernel_size=(3,3),padding="same",activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(62,activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32,activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(26,activation="softmax"),
])

In [19]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model.fit(x_train,y_train,epochs=10)

Epoch 1/10
376/376 [==============================] - 185s 487ms/step - loss: 0.2262 - accuracy: 0.0409
Epoch 2/10
376/376 [==============================] - 300s 798ms/step - loss: 0.1667 - accuracy: 0.0874
Epoch 3/10
376/376 [==============================] - 254s 675ms/step - loss: 0.1451 - accuracy: 0.1803
Epoch 4/10
376/376 [==============================] - 442s 1s/step - loss: 0.1301 - accuracy: 0.2607
Epoch 5/10
245/376 [==================>...........] - ETA: 2:43 - loss: 0.1208 - accuracy: 0.3103

In [147]:
model.evaluate(x_test,y_test)

94/94 [==============================] - 1s 9ms/step - loss: 0.0534 - accuracy: 0.7831


[0.05336456000804901, 0.7831004858016968]

In [148]:
y_pre=model.predict(x_test)

94/94 [==============================] - 1s 5ms/step


In [149]:
len(y_pre)

3006

In [150]:
len(y_test)

3006

In [151]:
y_predic=[]
for i in range(len(y_test)):
  y_predic.append(np.argmax(y_pre[i]))

In [152]:
y_predicte=np.array(y_predic)

In [153]:
y_predicted=tf.keras.utils.to_categorical(y_predicte,26,dtype="float32")

In [154]:
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [155]:
y_predicted[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [156]:
from sklearn.metrics import confusion_matrix,classification_report
r=classification_report(y_test,y_predicted)

In [157]:
print(r)

              precision    recall  f1-score   support

           0       0.97      0.92      0.94       110
           1       0.67      0.64      0.66       138
           2       0.79      0.88      0.83       106
           3       0.62      0.62      0.62       104
           4       0.76      0.76      0.76       115
           5       0.23      0.35      0.28       110
           6       0.51      0.42      0.46       108
           7       0.31      0.41      0.35       111
           8       0.98      0.99      0.99       109
           9       0.95      0.92      0.94       112
          10       0.95      0.79      0.86       139
          11       0.98      0.87      0.92        92
          12       0.62      0.64      0.63       105
          13       0.80      0.91      0.85       117
          14       0.86      0.88      0.87       118
          15       0.96      0.87      0.92       118
          16       0.96      0.90      0.93       105
          17       0.90    

In [158]:
model.save("Vegetable_CNN")

In [159]:
!zip -r Vegetable_CNN.zip /content/Vegetable_CNN

  adding: content/Vegetable_CNN/ (stored 0%)
  adding: content/Vegetable_CNN/assets/ (stored 0%)
  adding: content/Vegetable_CNN/keras_metadata.pb (deflated 93%)
  adding: content/Vegetable_CNN/fingerprint.pb (stored 0%)
  adding: content/Vegetable_CNN/saved_model.pb (deflated 90%)
  adding: content/Vegetable_CNN/variables/ (stored 0%)
  adding: content/Vegetable_CNN/variables/variables.data-00000-of-00001 (deflated 28%)
  adding: content/Vegetable_CNN/variables/variables.index (deflated 65%)


In [160]:
ur="/tyson4.jpg"

In [161]:
dc=pathlib.Path(ur)

In [165]:
xc=cv2.imread(str(dc))

In [166]:
xc = np.expand_dims(xc, axis=0)
q=model.predict(xc)
print(q)

1/1 [==============================] - 1s 746ms/step
[[4.8756578e-20 1.0128852e-03 2.2968633e-13 8.8121880e-05 3.7420591e-07
  6.1292539e-04 1.9257632e-06 9.9820721e-01 0.0000000e+00 1.6366583e-20
  3.0086754e-15 8.4867020e-19 5.8545280e-09 1.4234781e-09 4.5379078e-09
  1.3507535e-13 2.9562394e-15 4.3173628e-08 4.3465727e-26 3.3196190e-16
  9.1649596e-12 2.0322379e-07 1.6108756e-05 4.6232555e-05 1.3902383e-05
  4.4941073e-17]]


In [167]:
vis=np.argmax(q)
for ip,jp in label_dic.items():
    if jp==vis:
      print(ip)
      break

Peas
